# First Steps Connecting to the Wereldculturen MS SQL Server

Using pyodbc to connect to the server and submit queries to the DB


Can also use Pandas to directly parse query result into DataFrame (via `pandas.read_sql`)

In [113]:
from tqdm import tqdm
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [3]:
import pyodbc 
server = 'tcp:azuredfserv.database.windows.net' 
database = 'Azuredf' 
username = 'Demouser' 
password = 'Knxdde#77' 
driver='{ODBC Driver 17 for SQL Server}'

In [13]:
def table_to_DataFrame(connection, table_name, until=None, random_n=None):
    if not until:
        until = ""
    until = f"TOP {until}" if until else ""
    sample = f"TABLESAMPLE ({random_n} ROWS)" if random_n else ""
    query = f"SELECT {until} * FROM {table_name} {sample};"
    print(query)
    df = pd.read_sql(query, connection)
    return df


def connect_to_DB():
    return pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

# 1. Get List of Tables in the Database

In [116]:
with connect_to_DB() as conn:
    q  = "SELECT t.name, t.modify_date FROM sys.tables t"
    tables = pd.read_sql(q, conn)
    tables = tables[tables.name != "Person"]
    print(tables)

                  name             modify_date
1  ClassificationXRefs 2021-02-05 11:09:55.290
2          Departments 2021-02-05 11:09:55.320
3              Objects 2021-02-05 11:09:55.323
4      Classifications 2021-02-05 11:09:55.330


## 1.1 Count Number of Rows in the Tables

In [117]:
# q = "SELECT COUNT(*) FROM ClassificationXRefs;"

# q = """select t.name TableName, i.rows Records
# from sysobjects t, sysindexes i
# where t.xtype = 'U' and i.id = t.id and i.indid in (0,1)
# order by TableName;
# """

# both (above and below) queries return the same output

q = """SELECT 
    T.NAME AS 'TABLE NAME',
    P.[ROWS] AS 'NO OF ROWS'
FROM SYS.TABLES T 
INNER JOIN  SYS.PARTITIONS P ON T.OBJECT_ID=P.OBJECT_ID"""


with connect_to_DB() as conn:
    with conn.cursor() as cursor:
        cursor.execute(q)
        print(cursor.fetchall())

[('Person', 0), ('ClassificationXRefs', 11844602), ('Departments', 429), ('Objects', 11845922), ('Classifications', 418)]


## 2. Get Tables

In [118]:
with connect_to_DB() as conn:
    tables = {t[1][0]: table_to_DataFrame(conn, t[1][0], random_n=500) for t in tqdm(tables.iterrows())}


0it [00:00, ?it/s]

SELECT  * FROM ClassificationXRefs TABLESAMPLE (500 ROWS);



1it [00:02,  2.02s/it]

SELECT  * FROM Departments TABLESAMPLE (500 ROWS);



2it [00:02,  1.00it/s]

SELECT  * FROM Objects TABLESAMPLE (500 ROWS);



3it [00:17,  7.36s/it]

SELECT  * FROM Classifications TABLESAMPLE (500 ROWS);



4it [00:17,  4.39s/it]


## 2.1 Compute Expected Maximal Memory Sizes of Full Tables 

In [119]:
for t, df in tables.items():
    print(t)
    print("\t", df.memory_usage(deep=True).sum()/df.shape[0]*1e6/1e6, "\tMB predicted")
    print()

ClassificationXRefs
	 158.71046770601336 	MB predicted

Departments
	 300.7086247086247 	MB predicted

Objects
	 2693.643717728055 	MB predicted

Classifications
	 438.51674641148327 	MB predicted



## 2.2 Column Names

In [120]:
for t, df in tables.items():
    print("\n", t)
    print("\t", df.columns)


 ClassificationXRefs
	 Index(['ClassificationXRefID', 'ClassificationID', 'ID', 'TableID', 'LoginID',
       'EnteredDate', 'DisplayOrder', 'GSRowVersion'],
      dtype='object')

 Departments
	 Index(['DepartmentID', 'Department', 'Mnemonic', 'InputID', 'LoginID',
       'EnteredDate', 'NumRandomObjs', 'DefaultFormID', 'MainTableID',
       'GSRowVersion'],
      dtype='object')

 Objects
	 Index(['ObjectID', 'ObjectNumber', 'SortNumber', 'ObjectCount', 'DepartmentID',
       'ObjectStatusID', 'ClassificationID', 'SubClassID', 'Type',
       'LoanClassID', 'DateBegin', 'DateEnd', 'ObjectName', 'Dated', 'Title',
       'Medium', 'Dimensions', 'Signed', 'Inscribed', 'Markings', 'CreditLine',
       'Chat', 'DimensionRemarks', 'Description', 'Exhibitions', 'Provenance',
       'PubReferences', 'Notes', 'CuratorialRemarks', 'RelatedWorks',
       'Portfolio', 'PublicAccess', 'CuratorApproved', 'OnView',
       'TextSearchID', 'LoginID', 'EnteredDate', 'Accountability',
       'PaperFileR

In [131]:
for t, df in tables.items():
    print(t, f" {df.shape} (rows, cols)")
    print(df.apply(lambda col: len(col.unique()), axis=0))
#     plt.show()
    print()

ClassificationXRefs  (449, 8) (rows, cols)
ClassificationXRefID    449
ClassificationID          9
ID                      449
TableID                   1
LoginID                   3
EnteredDate             114
DisplayOrder              1
GSRowVersion            449
dtype: int64

Departments  (429, 10) (rows, cols)
DepartmentID      39
Department        18
Mnemonic          18
InputID            1
LoginID            7
EnteredDate       32
NumRandomObjs      2
DefaultFormID      2
MainTableID        9
GSRowVersion     429
dtype: int64

Objects  (581, 81) (rows, cols)
ObjectID                581
ObjectNumber            581
SortNumber              581
ObjectCount              11
DepartmentID             13
                       ... 
SortSearchNumber        581
ConservationEntityID    581
WMID                     81
BeginISODate             72
EndISODate               85
Length: 81, dtype: int64

Classifications  (418, 11) (rows, cols)
ClassificationID       38
Classification          8
A

## 3. Construct Smaller DFs by Selecting Relevant Columns

### Classifications

| Field  | Meaning  | Values
|---|---|---|
| ClassificationID  | Used to link entries in `ClassificationXRefs`  | int, unique |
| Classification  | Actual class values | (string, 8 levels, most common 'Documentatie')  |
| AATCN |  Classification into types of objects | (string, 38 levels, uniform, e.g. 'Boeken') |
| SubClassification | Extra classifications | (string, 16 levels, most common 'Audiovisueel') |
| SubClassification2 | More extra classifications | (string, 14 levels, uniform, e.g. 'Drukwerk') |

<br>

 - `SubClassification3` is empty
 - idea: create tuple of `(Classification, SubClassification, SubClassification2)` as the single classification feature
 

### ClassificationXRefs

| Field  | Meaning  | Values
|---|---|---|
| ClassificationXRefID  | Only used in this table?  | int, unique |
| ClassificationID  | Used to link entries in `Classifications` | correspond to values of <br> `Classifications.Classification`, non-uniform counts  |
| ID |  Uned to link entries in `Objects` | correspond to values of `Objects.ID`, uniform |
| TableID | ID of this table (for other contexts) | single value: 108 |



 
### Departments

| Field  | Meaning  | Values
|---|---|---|
| DepartmentID  | Used to link entries in Objects  | int, unique |
| Department  | Actual department names | (int, 18 levels, most not assigned, others uniform)  |
| Mnemonic |  Shorthand for the departmant name (field Department) | (same as Department) |

<br>

 - is `MainTableID` the ID of the departments in the main table? -> would be useful in that case for unification


### Objects


| Field  | Meaning  | Values
|---|---|---|
| ObjectID  | Linked to entries in `ClassificationXRefs`  | int, unique |
| DepartmentID  | Linked to entries in `Departments` | corresponds to `Departments.DepartmentID`  |
| ClassificationID |  Linked to entries in `Classifications` <br>and `ClassificationXRefs` | corresponds to `ClassificationID` in both tables |
| ObjectName | Name of the type of object | (string, 163 levels, most common 'Foto') |
| Title | The object's title | string |
| Description | The object's description | string |
| Provenance | Description of the object's history | string |


<br>

 - `ObjectNumber` seems to be an external ID for objects (is unique, has prefixes such as "TM", "RV", "NL")
 - `SortNumber` is similar to `ObjectNumber`
 - what does `ObjectCount` indicate? does `ObjectCount > 1` imply that entries should be merged?
 - what do `DateBegin` and `DateEnd` refer to? (most objects have `DateEnd == DateBegin`, latest date is 1990)
 - same for `Dated` -> which date is this?
 - technical properties:
     - Medium is the object's material
     - Dimensions
     - Signed, Inscribed, Markings
     - CreditLine: by whome the object was given
 - Exhibitions: Title of the exhibition the object was displayed at
 - Provenance: Description of how the object was acquired by the museum
     
     



'PubReferences', 'Notes', 'CuratorialRemarks', 'RelatedWorks',
       'Portfolio', 'PublicAccess', 'CuratorApproved', 'OnView',
       'TextSearchID', 'LoginID', 'EnteredDate', 'Accountability',
       'PaperFileRef', 'ObjectLevelID', 'ObjectTypeID', 'ObjectScreenID',
       'UserNumber1', 'UserNumber2', 'UserNumber3', 'UserNumber4',
       'ObjectNameID', 'ObjectNameAltID', 'UserDate1', 'UserDate2',
       'UserDate3', 'UserDate4', 'State', 'CatRais', 'HistAttributions',
       'Bibliography', 'Negative', 'LoanClass', 'Edition', 'Cataloguer',
       'Curator', 'PaperSupport', 'IsVirtual', 'CatalogueISODate',
       'CuratorRevISODate', 'IsTemplate', 'ObjectNumber2', 'SortNumber2',
       'DateRemarks', 'DateEffectiveISODate', 'PhysicalParentID',
       'InJurisdiction', 'SearchObjectNumber', 'RMID', 'AMID', 'GSRowVersion',
       'SortSearchNumber', 'ConservationEntityID', 'WMID', 'BeginISODate',
       'EndISODate'













#### Other Notes

 - `EnteredDate` is in all tables, earliest values around 1995, not uniformly distributed

In [240]:
tables["Objects"].Provenance.value_counts() #.apply(lambda on: on.split("-")[0])

Bruikleen 1884                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   8
Pater Rafael Verbois, een misssionaris uit Scheut (Anderlecht) in Belgie, was actief in Jehol in Binnen Mongolie (nu Chengde in China) tussen 1910 en 1939           

In [209]:
list(tables["Objects"][tables["Objects"].ObjectCount == 2].Title)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '96. TATAUIERTER JÜNGLING.',
 'BILD 20. GESTALT EINER SAMOANERIN VON HINTEN.']

In [169]:
tables["Classifications"][tables["Classifications"].ClassificationID == 156]

,ClassificationID,Classification,AATID,AATCN,SourceID,LoginID,EnteredDate,SubClassification,SubClassification2,SubClassification3,GSRowVersion
36,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x01c\x9a\xa0'
62,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x01\x00\xa9\xf3'
100,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x01!\xd2\x94'
138,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x01B\xa9e'
188,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x00\x19\xac\x93'
226,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x00:\xaa\xb6'
264,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x00[\xa8\xd5'
302,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x00|\xa6\xf4'
340,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x00\x9d\xa5\x14'
383,156,Materiële cultuurcollectie,NaN,Voorwerpen,None,xmarjob,2005-03-02 13:01:27.290,None,None,None,b'\x00\x00\x00\x00\x00\xbe\xa3\x0c'


In [147]:
# tables["Classifications"]
for df in tables.values():
    print(df.EnteredDate.value_counts())
    print()

2013-03-14 14:31:58.473    117
2017-03-24 15:11:18.000    117
2014-07-07 15:29:33.970     22
2014-07-07 15:29:33.980     21
2014-07-07 15:29:33.977     16
                          ... 
2018-07-02 16:51:54.343      1
2018-07-02 16:04:26.890      1
2018-07-02 16:57:10.930      1
2018-07-02 16:04:26.820      1
2018-07-02 16:04:26.853      1
Name: EnteredDate, Length: 114, dtype: int64

2000-10-04 21:25:21.957    77
2014-06-04 12:38:16.330    22
2014-07-12 08:21:31.713    11
2010-07-01 13:14:42.680    11
2010-07-01 13:14:42.663    11
2010-07-01 13:14:42.710    11
2005-03-02 13:18:23.280    11
2018-09-25 16:16:38.587    11
2017-08-15 06:58:13.463    11
2018-04-19 13:24:57.280    11
2018-10-02 14:51:29.007    11
2014-07-07 12:21:17.173    11
2014-07-12 08:22:20.253    11
2016-07-09 00:46:37.467    11
2005-03-02 09:44:43.723    11
1999-12-17 15:10:33.623    11
2010-07-01 13:14:42.657    11
2010-07-01 13:14:42.620    11
2016-07-09 04:20:42.070    11
2000-10-04 21:25:21.967    11
2010-07-01 13

## 4. Link Tables and Construct Unified Structure



|   	|   	|   	|
|---	|---	|---	|
| `Classifications.ClassificationID` | <---> | `ClassificationXRefs.ClassificationID` |
|`ClassificationXRefs.ID` | <---> |`Objects.ObjectID`|
|`Objects.DepartmentID` | <---> | `Departments.DepartmentID`|

<br>

 - `[table].TableID` is used to resolve IDs across contexts (i.e. while a certain `ID` is likely not unique across tables, the tuple of `(TableID, ID)` is) -> LET OP: not all tables have a `TableID`




# 0. Make Connection and List Databases on the Server

In [29]:
with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT name, collation_name FROM sys.databases")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()
            

master SQL_Latin1_General_CP1_CI_AS
azuredf SQL_Latin1_General_CP1_CI_AS
